<a href="https://colab.research.google.com/github/LuisCastroLobo/ProyectoProgra/blob/main/ProyectoProgra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
Prueba= 1
Prueba

1